In [ ]:
pip install -r requirements.txt

# EDA for the default risk dataset

In this notebook we conduct first an EDA analyseois to better understand the default risk dataset. The goal is to understand what data is available, and we can use as features for a classification model. In the modelling notebook , we then train a KNN model to predict the default risk. 

You can all the required packages using the requirements.txt file in the repository as follows (from command line): 



## Read and clean data

The default dataset is contained in the dataset folder in the repository. We will read the data and clean it to make it ready for analysis. 

The following info is provided of the dataset by the company. 
